# Importando módulos 

In [1]:
import obspy
from obspy.taup import TauPyModel

from multiprocessing import Pool
from obspy import read,UTCDateTime,Trace
from obspy.clients.fdsn import Client
import os
import glob
import numpy as np
from collections import defaultdict
import pandas as pd

#para plotar as figuras
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition,inset_axes
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

from datetime import datetime,timedelta,date
from tqdm import tqdm

from shapely.geometry.polygon import LinearRing

import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from IPython.display import HTML
from IPython import display

# Inputs e Outputs

In [2]:
FOLDER_OUTPUT = '/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/'

MSEED_INPUT = "/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/MSEED/"

METADATA_OUTPUT = "/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/METADATA/"

filename_csv = '/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/gliders_data/info_csv/metadados_glider_acustico_pmpas-bs.csv'

# Extraindo informações da tabela com os novos nomes dos arquivos (.feather)

In [4]:
df_mergulho_feather = pd.read_feather('/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/NETTAB/df_campanha_glider.feather')

In [5]:
df_mergulho_feather

,time,filename_mseed,mergulho,date,lat,lon,dep,N/S,Fs,nBits,...,start_lat,end_lat,start_lon,end_lon,ciclo,modelo,hidrofone,sensor,preAmp,registrador
0,"[2015-11-12T13:48:40.999995, 2015-11-12T13:50:...","[pa0001au_001_151112_134841, pa0001au_002_1511...","[0001, 0001, 0001, 0001, 0002, 0002, 0002, 000...","[2015-11-12, 2015-11-12, 2015-11-12, 2015-11-1...","[-23.825053500000003, -23.8246425, -23.8242279...","[-42.495485, -42.4954925, -42.495501000000004,...","[4.36492065, 20.073999999999998, 37.1260000000...","[SG618, SG618, SG618, SG618, SG618, SG618, SG6...",125000,16,...,-23.825054,-25.079344,-42.495485,-42.272640,1,SG618,"[911006, 911006, 911006, 911006, 911006, 91100...",HTI_92_WB,"[1501, 1501, 1501, 1501, 1501, 1501, 1501, 150...",EOS_HM1
1,"[2016-01-08T10:14:23.000003, 2016-01-08T10:16:...","[pa0001au_001_160108_101423, pa0001au_002_1601...","[0001, 0001, 0001, 0001, 0002, 0002, 0002, 000...","[2016-01-08, 2016-01-08, 2016-01-08, 2016-01-0...","[-23.8396465, -23.8402455, -23.840843999999997...","[-42.50416, -42.504215, -42.5042725, -42.50433...","[4.4947145, 20.937999499999997, 37.24699850000...","[SG618, SG618, SG618, SG618, SG618, SG618, SG6...",125000,16,...,-23.839647,-24.353146,-42.504160,-42.316597,2,SG618,"[911006, 911006, 911006, 911006, 911006, 91100...",HTI_92_WB,"[1501, 1501, 1501, 1501, 1501, 1501, 1501, 150...",EOS_HM1
2,"[2016-02-03T00:00:57.000003, 2016-02-03T00:02:...","[pa0007au_001_160203_000057, pa0007au_002_1602...","[0007, 0007, 0007, 0007, 0007, 0007, 0007, 000...","[2016-02-03, 2016-02-03, 2016-02-03, 2016-02-0...","[-24.3285225, -24.3279755, -24.3273995, -24.32...","[-42.323730499999996, -42.323891, -42.324061, ...","[2.3261719000000003, 22.924609, 42.826563, 60....","[SG612, SG612, SG612, SG612, SG612, SG612, SG6...",125000,24,...,-24.328522,-24.465736,-42.323730,-43.148800,3,SG612,"[911008, 911008, 911008, 911008, 911008, 91100...",HTI_92_WB,"[1503, 1503, 1503, 1503, 1503, 1503, 1503, 150...",EOS_HM1
3,"[2016-07-08T17:20:07.999999, 2016-07-08T17:23:...","[160708_172008_pa0003au_001, 160708_172307_pa0...","[0003, 0003, 0003, 0003, 0003, 0003, 0003, 000...","[2016-07-08, 2016-07-08, 2016-07-08, 2016-07-0...","[-24.391225000000002, -24.391524333333333, -24...","[-43.821711, -43.82178233333334, -43.821897, -...","[7.286890366666666, 20.01364633333333, 40.3452...","[SG612, SG612, SG612, SG612, SG612, SG612, SG6...",125000,24,...,-24.391225,-24.784666,-43.821711,-43.082054,5,SG612,"[911008, 911008, 911008, 911008, 911008, 91100...",HTI_92_WB,"[1503, 1503, 1503, 1503, 1503, 1503, 1503, 150...",EOS_HM1
4,"[2016-08-17T07:45:40.999996, 2016-08-17T07:48:...","[160817_074541_pa0005au_001, 160817_074841_pa0...","[0005, 0005, 0005, 0005, 0005, 0005, 0005, 000...","[2016-08-17, 2016-08-17, 2016-08-17, 2016-08-1...","[-24.812242333333334, -24.812640666666667, -24...","[-43.140601333333336, -43.14077266666666, -43....","[7.344921800000001, 29.604296333333334, 55.430...","[SG612, SG612, SG612, SG612, SG612, SG612, SG6...",125000,24,...,-24.812242,-24.361103,-43.140601,-43.142606,6,SG612,"[911008, 911008, 911008, 911008, 911008, 91100...",HTI_92_WB,"[1503, 1503, 1503, 1503, 1503, 1503, 1503, 150...",EOS_HM1
5,"[2016-09-16T23:59:00.000004, 2016-09-17T00:01:...","[160916_235900_pa0003au_061, 160917_000159_pa0...","[0003, 0003, 0003, 0005, 0005, 0005, 0005, 000...","[2016-09-17, 2016-09-17, 2016-09-17, 2016-09-1...","[-24.36140633333333, -24.361825999999997, -24....","[-43.14263166666667, -43.14266466666666, -43.1...","[929.5903166666667, 945.9947333333333, 953.780...","[SG612, SG612, SG612, SG612, SG612, SG612, SG6...",125000,24,...,-24.361406,-24.541116,-43.142632,-43.646273,7,SG612,"[911008, 911008, 911008, 911008, 911008, 91100...",HTI_92_WB,"[1503, 1503, 1503, 1503, 1503, 1503, 1503, 150...",EOS_HM1
6,"[2016-10-21T09:07:15.000004, 2016-10-21T09:10:...","[161021_090715_pa0001au_001, 161021_091015_pa0...","[0001, 0001, 0001, 0001, 0008, 0008, 0008, 000...","[2016-10-21, 2016-10-21, 2016-10-21, 2016-10-2...","[-24

# Extraindo informações dos arquivos ".mseed"

In [6]:
filenames_MSEED = sorted(glob.glob(MSEED_INPUT+'*/*/*.mseed'))

In [20]:
def mergulho_start_end_time(k):
    try:
        
        files_mseed_HD = [i for i in filenames_MSEED if i.split('/')[-1].split('.')[0] in k[1]['filename_mseed'].tolist()]
        
        tr_start = obspy.read(files_mseed_HD[0],headonly=True)[0]

        tr_end = obspy.read(files_mseed_HD[-1],headonly=True)[0]
        
        filename_mseed_HD = [i.split('/')[-1].split('.')[0] for i in files_mseed_HD]
        df_file = k[1].to_frame()

        df_nettab = df_file.T
        df_nettab['date_start_nettab'] = tr_start.stats.starttime.strftime('%Y/%j:%H%M')
        df_nettab['date_end_nettab'] = tr_end.stats.endtime.strftime('%Y/%j:%H%M')
        df_nettab['filename_mseed_HD'] = [filename_mseed_HD]

        return df_nettab
    
    except:
       pass

In [21]:
end_time = []
with Pool(processes=4) as p:
    with tqdm(total=len(df_mergulho_feather)) as pbar:
        for result in p.imap_unordered(mergulho_start_end_time,df_mergulho_feather.iterrows()):
            pbar.update()
            end_time.append(result)

  0%|                                                                                                                                  | 0/45 [00:00<?, ?it/s]

Index(['time', 'filename_mseed', 'mergulho', 'date', 'lat', 'lon', 'dep',
       'N/S', 'Fs', 'nBits', 'Hidrofone', 'PreAmp', 'Ciclo', 'station_nettab',
       'date_start_nettab', 'date_end_nettab', 'filename_mseed_HD',
       'start_lat', 'end_lat', 'start_lon', 'end_lon', 'ciclo', 'modelo',
       'hidrofone', 'sensor', 'preAmp', 'registrador'],
      dtype='object')


  2%|██▋                                                                                                                       | 1/45 [00:04<03:37,  4.94s/it]

Index(['time', 'filename_mseed', 'mergulho', 'date', 'lat', 'lon', 'dep',
       'N/S', 'Fs', 'nBits', 'Hidrofone', 'PreAmp', 'Ciclo', 'station_nettab',
       'date_start_nettab', 'date_end_nettab', 'filename_mseed_HD',
       'start_lat', 'end_lat', 'start_lon', 'end_lon', 'ciclo', 'modelo',
       'hidrofone', 'sensor', 'preAmp', 'registrador'],
      dtype='object')


  4%|█████▍                                                                                                                    | 2/45 [00:09<03:34,  4.99s/it]

...

Index(['time', 'filename_mseed', 'mergulho', 'date', 'lat', 'lon', 'dep',
       'N/S', 'Fs', 'nBits', 'Hidrofone', 'PreAmp', 'Ciclo', 'station_nettab',
       'date_start_nettab', 'date_end_nettab', 'filename_mseed_HD',
       'start_lat', 'end_lat', 'start_lon', 'end_lon', 'ciclo', 'modelo',
       'hidrofone', 'sensor', 'preAmp', 'registrador'],
      dtype='object')


 49%|███████████████████████████████████████████████████████████▏                                                             | 22/45 [01:00<00:41,  1.79s/it]

Index(['time', 'filename_mseed', 'mergulho', 'date', 'lat', 'lon', 'dep',
       'N/S', 'Fs', 'nBits', 'Hidrofone', 'PreAmp', 'Ciclo', 'station_nettab',
       'date_start_nettab', 'date_end_nettab', 'filename_mseed_HD',
       'start_lat', 'end_lat', 'start_lon', 'end_lon', 'ciclo', 'modelo',
       'hidrofone', 'sensor', 'preAmp', 'registrador'],
      dtype='object')


 51%|█████████████████████████████████████████████████████████████▊                                                           | 23/45 [01:03<01:00,  2.75s/it]Process ForkPoolWorker-18:
Process ForkPoolWorker-17:
Process ForkPoolWorker-20:
Process ForkPoolWorker-19:
  File "/home/sysop/Programs/miniconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/sysop/Programs/miniconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/sysop/Programs/miniconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/sysop/Programs/miniconda3/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):



KeyboardInterrupt: 

In [9]:
dataframe_mseed_final = pd.concat(end_time, ignore_index=True)

In [26]:
dataframe_mseed_final

,time,filename_mseed,mergulho,date,lat,lon,dep,N/S,Fs,nBits,...,start_lat,end_lat,start_lon,end_lon,ciclo,modelo,hidrofone,sensor,preAmp,registrador
0,"[2016-01-08T10:14:23.000003, 2016-01-08T10:16:...","[pa0001au_001_160108_101423, pa0001au_002_1601...","[0001, 0001, 0001, 0001, 0002, 0002, 0002, 000...","[2016-01-08, 2016-01-08, 2016-01-08, 2016-01-0...","[-23.8396465, -23.8402455, -23.840843999999997...","[-42.50416, -42.504215, -42.5042725, -42.50433...","[4.4947145, 20.937999499999997, 37.24699850000...","[SG618, SG618, SG618, SG618, SG618, SG618, SG6...",125000,16,...,-23.839647,-24.353146,-42.50416,-42.316597,2,SG618,"[911006, 911006, 911006, 911006, 911006, 91100...",HTI_92_WB,"[1501, 1501, 1501, 1501, 1501, 1501, 1501, 150...",EOS_HM1
1,"[2016-07-08T17:20:07.999999, 2016-07-08T17:23:...","[160708_172008_pa0003au_001, 160708_172307_pa0...","[0003, 0003, 0003, 0003, 0003, 0003, 0003, 000...","[2016-07-08, 2016-07-08, 2016-07-08, 2016-07-0...","[-24.391225000000002, -24.391524333333333, -24...","[-43.821711, -43.82178233333334, -43.821897, -...","[7.286890366666666, 20.01364633333333, 40.3452...","[SG612, SG612, SG612, SG612, SG612, SG612, SG6...",125000,24,...,-24.391225,-24.784666,-43.821711,-43.082054,5,SG612,"[911008, 911008, 911008, 911008, 911008, 91100...",HTI_92_WB,"[1503, 1503, 1503, 1503, 1503, 1503, 1503, 150...",EOS_HM1
2,"[2016-08-17T07:45:40.999996, 2016-08-17T07:48:...","[160817_074541_pa0005au_001, 160817_074841_pa0...","[0005, 0005, 0005, 0005, 0005, 0005, 0005, 000...","[2016-08-17, 2016-08-17, 2016-08-17, 2016-08-1...","[-24.812242333333334, -24.812640666666667, -24...","[-43.140601333333336, -43.14077266666666, -43....","[7.344921800000001, 29.604296333333334, 55.430...","[SG612, SG612, SG612, SG612, SG612, SG612, SG6...",125000,24,...,-24.812242,-24.361103,-43.140601,-43.142606,6,SG612,"[911008, 911008, 911008, 911008, 911008, 91100...",HTI_92_WB,"[1503, 1503, 1503, 1503, 1503, 1503, 1503, 150...",EOS_HM1
3,"[2016-02-03T00:00:57.000003, 2016-02-03T00:02:...","[pa0007au_001_160203_000057, pa0007au_002_1602...","[0007, 0007, 0007, 0007, 0007, 0007, 0007, 000...","[2016-02-03, 2016-02-03, 2016-02-03, 2016-02-0...","[-24.3285225, -24.3279755, -24.3273995, -24.32...","[-42.323730499999996, -42.323891, -42.324061, ...","[2.3261719000000003, 22.924609, 42.826563, 60....","[SG612, SG612, SG612, SG612, SG612, SG612, SG6...",125000,24,...,-24.328522,-24.465736,-42.32373,-43.1488,3,SG612,"[911008, 911008, 911008, 911008, 911008, 91100...",HTI_92_WB,"[1503, 1503, 1503, 1503, 1503, 1503, 1503, 150...",EOS_HM1
4,"[2016-09-16T23:59:00.000004, 2016-09-17T00:01:...","[160916_235900_pa0003au_061, 160917_000159_pa0...","[0003, 0003, 0003, 0005, 0005, 0005, 0005, 000...","[2016-09-17, 2016-09-17, 2016-09-17, 2016-09-1...","[-24.36140633333333, -24.361825999999997, -24....","[-43.14263166666667, -43.14266466666666, -43.1...","[929.5903166666667, 945.9947333333333, 953.780...","[SG612, SG612, SG612, SG612, SG612, SG612, SG6...",125000,24,...,-24.361406,-24.541116,-43.142632,-43.646273,7,SG612,"[911008, 911008, 911008, 911008, 911008, 91100...",HTI_92_WB,"[1503, 1503, 1503, 1503, 1503, 1503, 1503, 150...",EOS_HM1
5,"[2015-11-12T13:48:40.999995, 2015-11-12T13:50:...","[pa0001au_001_151112_134841, pa0001au_002_1511...","[0001, 0001, 0001, 0001, 0002, 0002, 0002, 000...","[2015-11-12, 2015-11-12, 2015-11-12, 2015-11-1...","[-23.825053500000003, -23.8246425, -23.8242279...","[-42.495485, -42.4954925, -42.495501000000004,...","[4.36492065, 20.073999999999998, 37.1260000000...","[SG618, SG618, SG618, SG618, SG618, SG618, SG6...",125000,16,...,-23.825054,-25.079344,-42.495485,-42.27264,1,SG618,"[911006, 911006, 911006, 911006, 911006, 91100...",HTI_92_WB,"[1501, 1501, 1501, 1501, 1501, 1501, 1501, 150...",EOS_HM1
6,"[2016-10-21T09:07:15.000004, 2016-10-21T09:10:...","[161021_090715_pa0001au_001, 161021_091015_pa0...","[0001, 0001, 0001, 0001, 0008, 0008, 0008, 000...","[2016-10-21, 2016-10-21, 2016-10-21, 2016-10-2...","[-24.6691

In [111]:
mergulho_name = []
for i in dataframe_mseed_final.iterrows():
    mergulho_name.append(i[1]['mergulho'][1:])

In [13]:
lines_nettab = []
for i in dataframe_mseed_final.iterrows():
    # Station lines (one line per station/sensor/epoch) # code description datalogger%sn seismometer%sn # sampling orientation lat. lon. elev. depth. start end
    #Sl: UNAP "Uni-Iquique/Chile"   DM24%A1383 CMG-3ESP/60%T34622 100 ZNE -20.24393 -70.14041 0.0    0.0 2009/134 
       
    lines_nettab.append('Sl: '+i[1].station_nettab+' Glider:PMPAS-BS'+' SENSOR'+' '+'REGISTER'+' 100 Z '+str(i[1].lat)+' '+str(i[1].lon)+' 0 0 '+i[1].date_start_nettab+' '+i[1].date_end_nettab)

In [14]:
with open('/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/gliders_data/info_csv/df_mergulho_nettab.txt', 'w') as f:
    for line in lines_nettab:
        f.write(line)
        f.write('\n')

# Renomeando os arquivos ".mseed":

In [20]:
def rename_files(k):
    
    files_mseed_HD = k[1]['filename_mseed_HD']
    
    files_path_mseed_HD = [glob.glob(MSEED_INPUT+'*/*/*'+i+'.mseed') for i in files_mseed_HD]
    for y in files_path_mseed_HD:
        
        st = obspy.read(y[0])
        st[0].stats.network = 'GL'
        st[0].stats.station = k[1]['station_nettab']
        st[0].stats.location = k[1]['mergulho_nettab']
        st[0].stats.channel = 'HHH'

        #<SDSdir>/Year/NET/STA/CHAN.TYPE/NET.STA.LOC.CHAN.TYPE.YEAR.DAY
        #Saving in SDS structure        
        OUTPUT_STREAM = FOLDER_OUTPUT+'MSEED_GL_SDS/'+st[0].stats.starttime.strftime("%Y")+'/'+st[0].stats.network+'/'+st[0].stats.station+'/HHH.D/'
        os.makedirs(OUTPUT_STREAM,exist_ok=True)
        
        station_name_file = st[0].stats.network+'.'+st[0].stats.station+'..'+st[0].stats.channel+'.D.'+st[0].stats.starttime.strftime("%Y.%j.%H.%M.%S")+'.mseed'
        if os.path.exists(OUTPUT_STREAM+station_name_file) == False:
            st.write(OUTPUT_STREAM+station_name_file, format='MSEED')

In [21]:
with Pool(processes=4) as p:
    with tqdm(total=len(df_mergulho_feather)) as pbar:
        for result in p.imap_unordered(rename_files,dataframe_mseed_final.iterrows()):
            pbar.update()

 95%|███████████████████████████████████▏ | 3918/4118 [1:44:50<05:21,  1.61s/it]
